<a href="https://colab.research.google.com/github/KunalVaidya99/Image-Colorization-Using-Deep-Learning/blob/main/noGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

In [ ]:
import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import (Conv2D,Conv2DTranspose,UpSampling2D,MaxPooling2D,Input
                          ,Activation,Concatenate,BatchNormalization,Flatten,Reshape
                          ,Dense,RepeatVector)
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.models import Model
import numpy as np
from keras.preprocessing.image import load_img
from PIL import Image
from skimage.color import gray2rgb
import os
import random
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint

In [ ]:
class DataGenerator(keras.utils.Sequence):
 
  def __init__(self,directory,filenames,batch_size=32,dim=(256,256),n_channels=3,shuffle=True):
    
    self.dim=dim
    self.filenames=filenames
    self.batch_size=batch_size
    self.n_channels=n_channels
    self.shuffle=shuffle
    self.directory=directory
    self.on_epoch_end()
    
 
  def __len__(self):
    return int(np.floor(len(self.filenames)/self.batch_size))
 
  def __getitem__(self,index):
    
    batch_files = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
 
    
 
    X,Y,resized,gray = self.__data_generation(batch_files)
 
    return X,Y,resized,gray
 
  def on_epoch_end(self):
    if self.shuffle == True :
      np.random.shuffle(self.filenames)
 
  def __data_generation(self,batch_files):
 
    X = np.empty((self.batch_size,*self.dim,1))
    Y = np.empty((self.batch_size,*self.dim,self.n_channels))
    resized  = np.empty((self.batch_size,224,224,3))
    gray = np.empty((self.batch_size,224,224,3))
    count =0 
    
 
    for files in batch_files:
 
     
      image = Image.open(self.directory + "/" + files)
 
      if (np.array(image).shape==(256,256,3)) :
 
        g_image = image

        img = image.resize((224,224))
        resized[count] = np.array(img)/255
        g = np.array(img.convert("L"))
        gray[count] = gray2rgb(g)/255

 
        Y[count,] = (np.array(image))/255
 
        X[count,] = (np.expand_dims(np.array(g_image.convert("L")),axis=-1))/255
 
        count = count + 1
 
    return X,Y,resized,gray

In [ ]:
directory = "/content/gdrive/My Drive/val_256"
filenames = os.listdir(directory)
random.shuffle(filenames)
split_size = 0.1
train_files = filenames[int(split_size*len(filenames)):len(filenames)]
test_files = filenames[:int(split_size*len(filenames))]
TrainGenerator = DataGenerator(directory,train_files)
TestGenerator = DataGenerator(directory,test_files)
img_shape = (256,256,1)